# HW #4: Data Analysis of Building Energy Benchmarking Data


This project focuses on analyzing the City of Calgary’s Building Energy Benchmarking dataset as part of a data science assignment. The objective is to explore, preprocess, and visualize the dataset using a variety of Python tools and techniques. The analysis will involve data cleaning, the application of Regular Expressions (Regex) for text standardization, and the use of Pandas and NumPy for data manipulation and aggregation. Additionally, visualizations will be generated using Matplotlib to identify trends and insights. This process will help to better understand the relationships between building characteristics and energy performance in Calgary.

# Part 1: Data Cleaning and Preprocessing

## 1.1 Load and Inspect the Dataset
__1.1.1__ Load the dataset and display its shape, column names, and data types.

__1.1.2__  Identify and list the number of missing values in each column..

In [2]:
import pandas as pd

# 1.1.1 Load the dataset and display its shape, column names, and data types.
df = pd.read_csv("Building_Energy_Benchmarking.csv")
df.shape             #shape
df.columns           #Columns names
df.info              # data types 


# 1.1.2. Identify and list the number of missing values in each column..
number_missing_values = df.isnull().sum()    #Missing data for each column


########################## Code to display the output in a well-organized format #####################################
# 1.1.1 Load the dataset and display its shape, column names, and data types.
print('\n')
print((' '*10),'1.1.1. LOAD THE DATASET AND DISPLAY ITS SHAPE, COLUMNS, AND DATA TYPES.', '\n')
print(('_'*40), 'SHAPE OF THE df',('_'*40 ))
print('\n','The shape of the df is',df.shape[0],'rows and',df.shape[1],'columns','\n')
print(('_'*38), 'NAME OF EACH COLUMN',('_'*38 ), '\n')
print(df.columns)
print(('_'*36), 'DATA TYPE IN EACH COLUMN',('_'*36 ))
print(df.info(), '\n')
print('\n')

# 1.1.2. Identify and list the number of missing values in each column.
print((' '*13),'1.1.2. IDENTIFY AND LIST THE NUMBER OF MISSING VALUES IN EACH COLUMN')
print('_'*97)
print(number_missing_values)



           1.1.1. LOAD THE DATASET AND DISPLAY ITS SHAPE, COLUMNS, AND DATA TYPES. 

________________________________________ SHAPE OF THE df ________________________________________

 The shape of the df is 494 rows and 31 columns 

______________________________________ NAME OF EACH COLUMN ______________________________________ 

Index(['Property Id', 'Property Name', 'Address 1', 'City', 'Postal Code',
       'Province', 'Primary Property Type - Self Selected',
       'Number of Buildings', 'Year Built',
       'Property GFA - Self-Reported (m²)', 'ENERGY STAR Score',
       'Site Energy Use (GJ)', 'Weather Normalized Site Energy Use (GJ)',
       'Site EUI (GJ/m²)', 'Weather Normalized Site EUI (GJ/m²)',
       'Source Energy Use (GJ)', 'Weather Normalized Source Energy Use (GJ)',
       'Source EUI (GJ/m²)', 'Weather Normalized Source EUI (GJ/m²)',
       'Total GHG Emissions (Metric Tons CO2e)',
       'Total GHG Emissions Intensity (kgCO2e/m²)',
       'Direct GHG Emissions (M

## 1.2 Handling Missing Data
__1.2.1__ Drop columns with more than 40% missing values.


__1.2.2__ For numerical columns, fill missing values with the median of their respective column.

__1.2.3__ For categorical columns, fill missing values with the mode of their respective column.

In [4]:
import pandas as pd 
from sklearn.impute import SimpleImputer

# 1.2.1 Drop columns with more than 40% missing values.
percent_missing = df.isnull().mean() * 100                          #Missing data for each column
drop_columns = df.columns[(percent_missing/100) > 0.4].tolist()     #Identify columns or rows with more than 40% missing values
dfc = df.drop(drop_columns, axis=1)                                 #Drop columns in a new DF called dfc 



# 1.2.2 For numerical columns, fill missing values with the median of their respective column.
num_cols = dfc.select_dtypes(include=['float64', 'int64']).columns  #Identify numerical columns
median_imputer = SimpleImputer(strategy = 'median')                 #Establish with what to replace missing values
dfc[num_cols] = median_imputer.fit_transform(dfc[num_cols])         #Replace missing values with median
print(num_cols.isnull().mean() * 100, '\n')



# 1.2.3 For categorical columns, fill missing values with the mode of their respective column.
categor_cols = dfc.columns.difference(num_cols)                                  #Identify categorical columns != numerical columns
dfc[categor_cols] = dfc[categor_cols].apply(lambda x: x.fillna(x.mode()[0]))     #Replace with mode in the missing values 



########################## Code to display the output in a well-organized format #####################################
# 1.2.1 Drop columns with more than 40% missing values.
print((' '*27),'1.1.2. DROP COLUMNS WITH MORE THAN 40% MISSING VALUES', '\n')
print(('_'*26), 'PERCENTAGE OF MISSING DATA FOR EACH COLUMN',('_'*26))
print(percent_missing, '\n')
print(('_'*25), 'COLUMNS DROPED WITH MORE THAN 40% MISSING VALUES',('_'*25))
print(drop_columns)
print('_'*100, '\n')

# 1.2.2 For numerical columns, fill missing values with the median of their respective column.
print('\n',(' '*6), '1.2.2 FOR NUMERICAL COLUMNS, FILL MISSING VALUES WITH THE MEDIAN OF THEIR RESPECTIVE COLUMN', '\n')
print(('_'*15), 'PERCENTAGE OF MISSING DATA IN EACH NUMERICAL COLUMN AFTER IMPUTATION',('_'*15))
print(dfc[num_cols].isnull().mean() * 100)
print('_'*100, '\n')

# 1.2.3 For categorical columns, fill missing values with the mode of their respective column.
print((' '*6), ' 1.2.3 FOR CATEGORICAL COLUMNS, FILL MISSING VALUES WITH THE MODE OF THEIR RESPECTIVE COLUMN', '\n')
print(('_'*14), 'PERCENTAGE OF MISSING DATA IN EACH CATEGORICAL COLUMN AFTER IMPUTATION',('_'*14))
print(dfc[categor_cols].isnull().mean() * 100)
print('_'*100, '\n')

0.0 

                            1.1.2. DROP COLUMNS WITH MORE THAN 40% MISSING VALUES 

__________________________ PERCENTAGE OF MISSING DATA FOR EACH COLUMN __________________________
Property Id                                                               0.000000
Property Name                                                             0.000000
Address 1                                                                 0.000000
City                                                                      0.000000
Postal Code                                                               0.000000
Province                                                                  0.000000
Primary Property Type - Self Selected                                     0.000000
Number of Buildings                                                       0.000000
Year Built                                                                0.000000
Property GFA - Self-Reported (m²)                                 

## 1.3 Extracting and Cleaning Data Using Regex
• **Use Regex only to:**

__1.3.1__ Extract numeric values from text-based numeric columns (e.g., Property GFA,
Energy Use, Emissions).

__1.3.2__ Standardize Postal Codes to follow the Canadian format (A1A 1A1).

__1.3.3__ Clean and extract meaningful text from Property Names and Addresses.

__1.3.4__ Ensure extracted values are properly converted to numerical types for analysis.

In [10]:
import re
import pandas as pd

#1.3.1 Extract numeric values from text-based numeric columns (e.g., Property GFA, Energy Use, Emissions).
def str_num(df):                                                                      #Function to find object columns with numerical value
    numeric_columns = []                                                              #List of the column names 'object with nuemerical value'
    for col in df.select_dtypes(include=['object']).columns:                          #Inlude only 'object' columns
        if df[col].astype(str).str.match(r'^-?\d+(\.\d+)?$', na=False).any():         #Identify if they actually contain numeric values   
            numeric_columns.append(col)                                               
    return numeric_columns
names = str_num(dfc)    
for col in names:                                                                    #Extract the numerical values from 'object' columns
    dfc[col] = dfc[col].apply(lambda x: float(re.findall(r'-?\d+\.?\d*', str(x))[0]) if re.findall(r'-?\d+\.?\d*', str(x)) else None)



# 1.3.2 Standardize Postal Codes to follow the Canadian format (A1A 1A1).
def Canadian_format(postal):                                                                   #Function to orginice the postal code with the Canadian format
    match = re.match(r'([a-zA-Z]\d[a-zA-Z])\s?(\d[a-zA-Z]\d)', str(postal).upper().strip())    #Not only the format as well the upper case  
    return f"{match.group(1)} {match.group(2)}" if match else None                       
dfc["Postal Code"] = dfc["Postal Code"].apply(Canadian_format)                                  #Apply the function to the column 'Postal Code'



# 1.3.3 Clean and extract meaningful text from Property Names and Addresses.
#Property Name: Accepts letters and numbers. Also characters like - and #. Does not accept double spacing
dfc['Property Name'] = dfc['Property Name'].apply(lambda x: re.sub(r'/^(?!.*  )[a-zA-Z0-9#-]+( [a-zA-Z0-9#-]+)?$/', '', x).strip())

#Addres 1: Function so that words with more than 3 letters (name) are not all capitalized but only the first letter of the word.
def capitalize_large_words(text):
    return re.sub(r'\b[A-Z]{3,}\b', lambda m: m.group(0).capitalize(), text)
dfc['Address 1']= dfc['Address 1'].apply(capitalize_large_words)


# 1.3.4 Ensure extracted values are properly converted to numerical types for analysis.
#dfc.info()

########################## Code to display the output in a well-organized format #####################################
#1.3.1 Extract numeric values from text-based numeric columns (e.g., Property GFA, Energy Use, Emissions)

# Detectar columnas con números almacenados como texto
print((' '*3),'1.3.1 EXTRACT NUMERIC VALUES FROM TEXT-BASED NUMERIC COLUMNS (E.G., PROPERTY GFA, ENERGY USE, EMISSIONS)', '\n')
print(('_'*26), 'list of *object column* names with numeric values',('_'*26))
print(names, '\n')
print('_'*100, '\n')

# 1.3.4 Ensure extracted values are properly converted to numerical types for analysis.

print((' '*6), '1.3.4 ENSURE EXTRACTED VALUES ARE PROPERLY CONVERTED TO NUMERICAL TYPES FOR ANALYSIS', '\n')
print(dfc.info())
print('_'*100, '\n')


# 1.3.2 Standardize Postal Codes to follow the Canadian format (A1A 1A1).
print('\n',(' '*6), '1.3.2 STANDARDIZE POSTAL CODES TO FOLLOW THE CANADIAN FORMAT (A1A 1A1).', '\n')
print(dfc["Postal Code"].head())
print('_'*100, '\n')

# 1.3.3 Clean and extract meaningful text from Property Names and Addresses.
print((' '*6), '1.3.3 CLEAN AND EXTRACT MEANINGFUL TEXT FROM PROPERTY NAMES AND ADDRESSES.', '\n')
dfc[['Property Name', 'Address 1']].head(20)

    1.3.1 EXTRACT NUMERIC VALUES FROM TEXT-BASED NUMERIC COLUMNS (E.G., PROPERTY GFA, ENERGY USE, EMISSIONS) 

__________________________ list of *object column* names with numeric values __________________________
['Property GFA - Self-Reported (m²)', 'Site Energy Use (GJ)', 'Weather Normalized Site Energy Use (GJ)', 'Source Energy Use (GJ)', 'Weather Normalized Source Energy Use (GJ)', 'Total GHG Emissions (Metric Tons CO2e)', 'Direct GHG Emissions (Metric Tons CO2e)', 'Electricity Use - Grid Purchase (kWh)', 'Natural Gas Use (GJ)'] 

____________________________________________________________________________________________________ 

       1.3.4 ENSURE EXTRACTED VALUES ARE PROPERLY CONVERTED TO NUMERICAL TYPES FOR ANALYSIS 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494 entries, 0 to 493
Data columns (total 26 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Prop

,Property Name,Address 1
0,Acadia Aquatic & Fitness Centre,9009 Fairmount Dr SE
1,Ad Valorem,2924 11 ST NE
2,Alberta Trade Centre,315 10 AV SE
3,Andrew Davison,133 6 AV SE
4,Animal Services Centre,2201 Portland ST SE
5,Apparatus Repair Shop and Spare Apparatus Shop,1725 18 AV NE
6,Beltline Aquatic & Fitness Centre,221 12 Av SW
7,Bob Bohan Aquatic and Fitness Centre,4812 14 Av SE
8,Bowmont Civic Building,5000 Bowness Civic Building
9,Calgary Public Building,205 8 AV SE


# Part 2: Exploratory Data Analysis (EDA) and Aggregations
## 2.1 Statistical Summary
__2.1.1__ Generate summary statistics for numerical features using extracted data.

__2.1.2__  Identify and explain key observations (e.g., outliers, mean vs. median differences).

In [20]:
import re
import pandas as pd

# 2.1.1 Generate summary statistics for numerical features using extracted data.
num_cols = dfc.select_dtypes(include=['float64', 'int64']).columns                             #Identify numerical columns
sum_stat = dfc[num_cols[4:-1]].describe()                                                      #Exclude columns with irrelevant numbers (e.g. Property ID)

# 2.1.2 Identify and explain key observations (e.g., outliers, mean vs. median differences)
# Calculate mean vs median differences
sum_stat.loc['mean vs median differences'] = sum_stat.loc['mean'] - dfc[num_cols].median()             #Add a new row with mean vs median
sum_stat.loc['mean_vs_median_labels'] = sum_stat.loc['mean vs median differences'].apply(lambda diff:  #Add new row describing the difference
                                                             'Right-skewed' if diff > 1 
                                                             else 'Left-skewed' if diff < 0 
                                                             else 'Almost Symmetric' if 0 < diff <= 1 
                                                             else 'Symmetric'
                                             )

#Code to extract outliers
IQR = sum_stat.loc['75%'] - sum_stat.loc['25%']                                                                                                
sum_stat.loc['Lower outlier'] = sum_stat.loc['25%'] - 1.5 *IQR       #lower outlier
sum_stat.loc['Upper outlier'] = sum_stat.loc['75%'] + 1.5 *IQR       #Upper outliers  


# I converted all values to integers as the values are very large.
sum_stat = sum_stat.apply(lambda col: col.map(lambda x: int(x) if isinstance(x, (int, float)) else x))  
sum_stat

,Site Energy Use (GJ),Weather Normalized Site Energy Use (GJ),Site EUI (GJ/m²),Weather Normalized Site EUI (GJ/m²),Source Energy Use (GJ),Weather Normalized Source Energy Use (GJ),Source EUI (GJ/m²),Weather Normalized Source EUI (GJ/m²),Total GHG Emissions (Metric Tons CO2e),Total GHG Emissions Intensity (kgCO2e/m²),Direct GHG Emissions (Metric Tons CO2e),Direct GHG Emissions Intensity (kgCO2e/m²),Electricity Use - Grid Purchase (kWh),Natural Gas Use (GJ)
count,494,494,494,494,494,494,494,494,494,494,494,494,494,494
mean,3586,3699,1,1,4556,4160,2,2,442,158,213,63,253922,2526
std,15596,15776,1,1,18232,17313,1,1,1078,109,713,56,834877,13849
min,1,1,0,0,1,1,0,0,1,10,0,0,1,1
25%,3,3,0,1,4,4,1,1,77,93,41,33,113,4
50%,79,81,1,1,36,36,1,1,173,117,74,43,462,472
75%,1558,1677,2,2,2079,1963,2,2,446,186,223,74,149170,1034
max,243202,242611,8,8,261481,260399,10,10,13067,666,12243,386,9618602,238415
mean vs median differences,3507,3618,0,0,4520,4124,0,0,269,41,139,19,253460,2053
mean_vs_median_labels,Right-skewed,Right-skewed,Almost Symmetric,Almost Symmetric,Right-skewed,Right-skewed,Almost Symmetric,Almost Symmetric,Right-skewed,Right-skewed,Right-skewed,Right-skewed,Right-skewed,Right-skewed


In [137]:
# 2.1.2 Identify and explain key observations (e.g., outliers, mean vs. median differences)
print(sum_stat.loc['mean vs median differences'].idxmin( ))
print(sum_stat.loc['mean vs median differences'].min( ))
print(sum_stat.loc['mean vs median differences'].idxmax( ))
print(sum_stat.loc['mean vs median differences'].max( ))
print(sum_stat.loc['std'].idxmax( ))
print(sum_stat.loc['std'].max( ))


Site EUI (GJ/m²)
0.48479757085020236
Electricity Use - Grid Purchase (kWh)
253460.66194331984
Electricity Use - Grid Purchase (kWh)
834877.8290134137
